In [ ]:
# !pip install trulens trulens-provider-openai openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
import pandas as pd
from trulens.core import Tru

tru = Tru()
tru.reset_database()

In [ ]:
data = {
    "query": ["hello world", "who is the president?", "what is AI?"],
    "query_id": ["1", "2", "3"],
    "expected_response": ["greeting", "Joe Biden", "Artificial Intelligence"],
    "expected_chunks": [
        None,
        None,
        None,
    ],  # This can also be lists if applicable
    "meta": [None, None, None],  # Metadata can also be a list of dictionaries
}

df = pd.DataFrame(data)

In [ ]:
tru.create_or_add_ground_truth_dataset(
    dataset_name="test_dataset_new", ground_truth_df=df
)